In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=13)


<IB connected to 127.0.0.1:7496 clientId=13>

In [2]:
from data import historical_data as hd
from frame.frame import Frame


class MarketsX:
    def __init__(self, ib):
        self.ib = ib
        self.frames = {
            '1 day': {},
            '1 hour': {},
            '5 mins': {},
        }
        
        self.sector_etfs = {
            'Technology': 'XLK',
            # 'Financials': 'XLF',
            # 'Communication Services': 'XLC',
            # 'Healthcare': 'XLV',
            # 'Consumer_Staples': 'XLP',
            # 'Consumer_Discretionary': 'XLY',
            # 'Energy': 'XLE',  
            # 'Industrials': 'XLI',  
            # 'Materials': 'XLB',  
            # 'Utilities': 'XLU',
            # 'Real_Estate': 'XLRE'
        }

    
    def setup_frames(self, timeframe,  start_date:str="52 weeksAgo", end_date:str='now', force_download:bool=False):
        if timeframe not in self.frames:
            raise ValueError(f"Invalid timeframe: {timeframe}. Choose from: {list(self.frames.keys())}")
        # load SPY first
        ohlcv = hd.get_hist_data('SPY', start_date, end_date, timeframe, force_download=force_download)
        self.frames[timeframe]['SPY'] = Frame('SPY', ohlcv, run_ta_on_load=True, rowHeights=[0.1, 0.1, 0.1, 0.1, 0.1, 0.5], name='S&P 500')
        print('MarketsX: SPY Frame loaded')

        # load sector ETFs
        for sector, symbol in self.sector_etfs.items():
            ohlcv = hd.get_hist_data(symbol, start_date, end_date, timeframe, force_download=force_download)
            self.frames[timeframe][symbol] = Frame(symbol, ohlcv, run_ta_on_load=True, rowHeights=[0.1, 0.1, 0.1, 0.1, 0.1, 0.5], name=sector)
        print('MarketsX: Sector Frames loaded')

    
    def get_frame(self, symbol, timeframe):
        if timeframe not in self.frames:
            raise ValueError(f"Invalid timeframe: {timeframe}. Choose from: {list(self.frames.keys())}")
        if symbol not in self.frames[timeframe]:
            raise ValueError(f"Symbol {symbol} not found in {timeframe} frames")
        return self.frames[timeframe][symbol]
    

        


mx = MarketsX(ib)
mx.setup_frames('1 day', force_download=False)

Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\SPY_1_day.csv
Stored data: 281 rows of data
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\SPY_1_day.csv
MarketsX: SPY Frame loaded
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\XLK_1_day.csv
Stored data: 281 rows of data
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\XLK_1_day.csv
MarketsX: Sector Frames loaded


In [8]:
import strategies.ta as ta
import strategies.signals as sig
import strategies.preset_strats as ps


def setup_ta(self, symbol, timeframe):
    spy = self.get_frame(symbol, timeframe)
    ps.require_ta_for_all(spy)
    ps.ma_ta(spy, [50, 150, 200])

    sector_frames = [f for f in self.frames[timeframe].values() if f.symbol != 'SPY']

    for f in sector_frames:
        """ Notes: 
        Required Sector Conditions:
        1. Mansfield RSI > 0 (close)
        2. Mansfiled RSI > 0 (volume)
        5. 50 MA upwards

        Bonus Sector Conditions:
        1. Breaks 50 MA 
        2. Volume > 10 MA (can choose premarket or regular hours)
        3. Room to Move
        4. Gap Up
        5. Breaks Res 1 
        """
        lookBack = 50
        f.import_data(spy.data, importCols=['close', 'volume'], prefix='SPY_')
        ps.require_ta_for_all(f, atrSpan=50)
        ps.ma_ta(f, [50, 150, 200])
        
        # Required Sector Conditions
        f.add_ta(ta.MansfieldRSI(stockCol='close', marketCol='SPY_close',   span=14), {'dash': 'solid', 'color': 'yellow', 'width': 1}, chart_type='line', row=5)
        f.add_ta(ta.MansfieldRSI(stockCol='volume', marketCol='SPY_volume', span=14), {'dash': 'solid', 'color': 'cyan', 'width': 1}, chart_type='line', row=5)
        f.add_ta(ta.PctChange(metric_column='MA_cl_50'), {'dash': 'solid', 'color': 'yellow', 'width': 1}, chart_type='line', row=3)
        f.add_ta(sig.Score(name='SectorValid', cols=['MRSI_14_SPY_close', 'MRSI_14_SPY_volume', 'MA_cl_50'], scoreType='all_gt', validThreshold=0, weight=1, lookBack=lookBack, normRange=(0,1)), {'dash': 'solid', 'color': 'yellow', 'width': 2}, chart_type='line', row=5)
        
        # Bonus Sector Conditions
        f.add_ta(sig.Breaks(price_column='close', direction='above', metric_column='MA_cl_50', normRange=(0,1), lookBack=lookBack), {'dash': 'solid', 'color': 'yellow', 'width': 1}, chart_type='line', row=3)
        ps.volume_ta(f, ls='LONG', ma=10, scoreRow=4, lookBack=lookBack)
        f.add_ta(sig.Breaks(price_column='close', direction='above', metric_column='Res_1_Upper', normRange=(0,1), lookBack=lookBack), {'dash': 'solid', 'color': 'yellow', 'width': 1}, chart_type='line', row=3)
        f.add_ta(sig.RoomToMove(tgetCol='Res_1_Lower'), {'dash': 'solid', 'color': 'yellow', 'width': 1}, chart_type='line', row=3)
        f.add_ta(sig.GapSize(atrCol='ATR_50', pointCol='HP_hi_10'), {'dash': 'solid', 'color': 'yellow', 'width': 1}, chart_type='line', row=3)
        # todo: add overall score for the Bonus Sector Conditions


setup_ta(mx, 'SPY', '1 day')
mx.get_frame('SPY', '1 day').plot()
mx.get_frame('XLK', '1 day').plot()
        

In [5]:

display(mx.get_frame('XLK', '1 day').data)


# setup_ta(mx, 'XLK', '1 day')
mx.get_frame('XLK', '1 day').plot()
mx.get_frame('SPY', '1 day').plot()

,open,high,low,close,volume,SPY_close,SPY_volume,ATR_50,HP_hi_10,LP_lo_10,...,MA_cl_200,MA_vo_10,SigL_VolSpike,SigL_VolROC,Score_L_Vol,MRSI_14_SPY_close,MRSI_14_SPY_volume,BRK_close_ab_MA_cl_50,PCT_MA_cl_50_1,Score_MRSIValid
date,,,,,,,,,,,,,,,,,,,,,
2024-01-23,200.88,202.19,200.07,202.15,51263,485.97,376837,2.1200,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
2024-01-24,203.12,205.29,202.67,203.47,68453,485.47,644598,2.3700,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.037690,-1.231891,NaN,NaN,NaN
2024-01-25,203.96,206.15,202.40,202.44,67291,486.94,564435,2.8300,206.15,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.028889,-0.104943,NaN,NaN,NaN
2024-01-26,202.26,203.57,201.29,201.71,41356,486.75,516378,2.6925,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.045795,-2.744557,NaN,NaN,NaN
2024-01-29,202.39,204.19,201.60,204.18,30053,491.29,450317,2.6720,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.013617,-3.434899,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-15,228.00,232.97,227.77,232.77,40269,593.66,388542,3.6238,NaN,NaN,...,221.52450,33230.8,10.59,18.18,18.18,-0.121872,0.496412,0.0,0.084707,0.0
2025-01-16,234.44,234.59,230.00,230.22,18658,591.51,245768,3.6476,NaN,NaN,...,221.63100,32512.0,0.00,0.00,0.00,-0.175466,-2.152941,0.0,0.048071,0.0
2025-01-17,232.00,235.46,231.50,234.23,23470,597.69,312023,3.6756,NaN,NaN,...,221.76895,33415.7,0.00,8.60,8.60,-0.092753,-1.886286,0.0,0.088599,0.0


In [28]:
mx.get_frame('XLK', '1 day').data.columns

Index(['open', 'high', 'low', 'close', 'volume', 'MRSI_14_SPY', 'ATR_50',
       'HP_hi_10', 'LP_lo_10', 'Res_1', 'Res_1_Upper', 'Res_1_Lower', 'Res_2',
       'Res_2_Upper', 'Res_2_Lower', 'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower',
       'Sup_2', 'Sup_2_Upper', 'Sup_2_Lower', 'MA_cl_50', 'MA_cl_150',
       'MA_cl_200', 'MA_vo_10', 'SigL_VolSpike', 'SigL_VolROC', 'Score_L_Vol',
       'BRK_close_ab_MA_cl_50', 'SPY_close'],
      dtype='object')